In [1]:
import numpy as np
import pandas as pd

import pickle
from congress import Congress
import tweepy

In [2]:
pkl_file = open('/home/polichinel/Dropbox/credential/essentials.pkl', 'rb')
essentials = pickle.load(pkl_file)

consumer_key = essentials["twitter_consumer_key"]
consumer_secret = essentials["twitter_consumer_secret"]

access_token = essentials["twitter_Access_Token"]
access_secret = essentials["twittet_Access_Token_Secret"]

auth = tweepy.OAuthHandler(consumer_key=consumer_key, consumer_secret=consumer_secret)
auth.set_access_token(key=access_token, secret=access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify = True)

congress = Congress(essentials["congress_key"])


pkl_file.close()

In [3]:
senate = congress.members.filter("senate")
house = congress.members.filter("house")

senate_members = senate[0]['members']
house_members = house[0]['members']

df_senata = pd.DataFrame(senate_members)
df_house = pd.DataFrame(house_members)


legs = df_house.twitter_account
legs = legs.append(df_senata.twitter_account)

legs.dropna(inplace=True)

In [4]:
total_n = len(legs) # gives three more then before.
print(total_n)

538


In [7]:
# the scraping -----------------------------------------------------------------------

time_lines = []
x=0 # crahsed at 296

for legislator in legs[0:5]: # set number if needed - last crash at 296
    x=x+1
    print(str(x) + "/" + str(total_n), end=' ') # printing horizontal "loadingline"
    
    #crash backup
    file_name = "/media/polichinel/DATA/backup/PDS_DATA/all_timelines/time_lines" + str(x) + ".pkl"
    output = open(file_name, 'wb') # still not working right!!
    pickle.dump(time_lines, output)
    output.close()

    try:
        for tweet in tweepy.Cursor(api.user_timeline, id=legislator).items(100): # how many tweets from each pol - blank means to limit rata
            time_lines.append(tweet)
            
    except(RuntimeError, TypeError, NameError, ZeroDivisionError, IndexError, tweepy.TweepError)  as e:
        print(e, end=' ')
        continue


1/538 2/538 3/538 4/538 5/538 

In [ ]:
len(time_lines)

In [ ]:
# IS it safe?
# output = open('time_lines.pkl', 'wb')
# pickle.dump(time_lines, output)
# output.close()

In [ ]:
# Making the dataframe

def process_one_user(time_lines):
    id_list = [tweet.id for tweet in time_lines]
    data_set = pd.DataFrame(id_list, columns=["id"])

    # Processing Tweet Data

    data_set["text"] = [tweet.text for tweet in time_lines]
    data_set["created_at"] = [tweet.created_at for tweet in time_lines]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in time_lines]
    data_set["favorite_count"] = [tweet.favorite_count for tweet in time_lines]
    data_set["source"] = [tweet.source for tweet in time_lines]

    # Processing User Data
    data_set["user_id"] = [tweet.author.id for tweet in time_lines]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in time_lines]
    data_set["user_name"] = [tweet.author.name for tweet in time_lines]
    data_set["user_created_at"] = [tweet.author.created_at for tweet in time_lines]
    data_set["user_description"] = [tweet.author.description for tweet in time_lines]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in time_lines]
    data_set["user_friends_count"] = [tweet.author.friends_count for tweet in time_lines]
    data_set["user_location"] = [tweet.author.location for tweet in time_lines]

    return data_set
data_set = process_one_user(time_lines)

In [ ]:
print("dimensions of dataframe: "  + str(data_set.shape))
data_set.head(6)

In [ ]:
data_set.tail(6)

In [ ]:
data_set.to_csv("new_new_many_tweets.csv")